In [ ]:
def convertir_a_centimos(precio_str):

    precio_str = precio_str.replace(",", ".")

    try:
        precio_float = float(precio_str)
    except ValueError:
        return None  
    precio_centimos = int(round(precio_float * 100))

    return precio_centimos


In [ ]:
from flask import Flask, jsonify
import  pandas as pd
import joblib
app=Flask(__name__)
@app.route('/grafica/<Precio>/<VRAM>/<HDMI>/<Displayport>/<Energia>/<GDDR>')
def get_ordenador(Precio, VRAM, HDMI, Displayport,Energia, GDDR):
   
    try:
        ggdr3 = ggdr4 = ggdr5 = ggdr6 = ggdr6x = ggdr7 = False
        precioValor2 = convertir_a_centimos(Precio)
        if precioValor2 is None:
            return jsonify({"error": "Precio inválido. Debe ser un número válido."}), 400
        if VRAM.isdigit():
            vramValor = VRAM
        if HDMI.isdigit():
            hdmiValor = HDMI
        if Displayport.isdigit():
            displayValor = Displayport
        if Energia.isdigit():
            energiaValor = Energia
        match GDDR:
            case "3":
                ggdr3 = True
            case "4":
                ggdr4 = True
            case "5":
                ggdr5 = True
            case "6":
                ggdr6 = True
            case "6x":
                ggdr6x = True
            case "7":
                ggdr7 = True
            case _:
                return jsonify({"error": "Tipo de GDDR no válido"}), 400
        
        datos = pd.DataFrame([{
            "VRAM": vramValor,
            "HDMI": hdmiValor,
            "DisplayPort": displayValor,
            "Energia": energiaValor,
            "GDDR3": ggdr3,
            "GDDR4": ggdr4,
            "GDDR5": ggdr5,
            "GDDR6": ggdr6,
            "GDDR6X": ggdr6x,
            "GDDR7": ggdr7
        }])

        modelo = joblib.load('modelo_gb.pkl')

        precio_predict =  modelo.predict(datos)

        if precio_predict[0]+(precio_predict[0]*0.15) < precioValor2:
            price_status =  "Mal precio muy por encima."
        elif precio_predict[0]+(precio_predict[0]*0.15) > precioValor2 and precio_predict[0] < precioValor2:
            price_status = "El precio esta un poco por encima."
        elif precio_predict[0]-(precio_predict[0]*0.20) < precioValor2 and precio_predict[0] > precioValor2:
            price_status = "El precio es bueno."
        else: 
            price_status = "Es muy buen precio."
        return jsonify({
            "Precio_Predicho": round(precio_predict[0], 0),
            "Precio_Real": precioValor2,
            "Estado": price_status
        })
    except Exception as e:
        return jsonify({"error": f"Ocurrió un error inesperado: {str(e)}"}), 500
    

if __name__=="__main__":
    app.run()   # localhost at 5000

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Mar/2025 22:53:07] "GET /grafica/610/16/2/2/750/7 HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2025 22:53:14] "GET /grafica/7444/16/2/2/750/7 HTTP/1.1" 200 -
